# Goal

The purpose of the script is to take two images and align them. The first image is the correct dimensions and print socks and orientation. The second image should be resized and rotated to align to a subsection of the first. The second image will have damaged pixels and lines and different color balances. It will also have one area that is white and very different than the image around it. Using the watershed air better algorithms find the outlines of this white area and change all the pixels within the outline to the clear alpha channel.  Use libraries like OpenCV (cv2) and Pillow (PIL) to isolate the alpha channel as a mask, then calculate transformations based only on non-zero alpha areas so that the second image is rotated and resized to best line with the first image. Lastly, take all of the pixels of the first image That are now overlapping with the alpha outlined region and generate a output image with those pixels and a sharp dark outline. Save this as a PNG with all of the metadata such that it will print out at the correct original scale. Provide an option to save the intermediate aligned images In a format that allows multiple layers.

This python script that can be used as a library, a command line tool, or a cross platform GUI with simple file loading dialogues.

In [1]:
pip install svgwrite

In [2]:
import cv2
import numpy as np
import os
from PIL import Image
import matplotlib.pyplot as plt
import svgwrite # Added for SVG output

class ImageAligner:
    """
    Library class for aligning images, isolating regions, and extracting composites.
    """
    def __init__(self, debug=False, detector=None, matcher=None):
        self.debug = debug

        # Initialize detector: SIFT as default as it worked well in tests
        if detector is None:
            self.detector = cv2.SIFT_create()
        else:
            self.detector = detector

        # Initialize matcher: BFMatcher with NORM_L2 as default for SIFT
        if matcher is None:
            self.matcher = cv2.BFMatcher(cv2.NORM_L2, crossCheck=False) # crossCheck=False for knnMatch with ratio test
        else:
            self.matcher = matcher

    def _find_white_region(self, img_bgra):
        """
        Finds the largest white region, ignoring noise and damaged lines.
        Returns a binary mask of the white area.
        """
        hsv = cv2.cvtColor(img_bgra, cv2.COLOR_BGRA2BGR)
        hsv = cv2.cvtColor(hsv, cv2.COLOR_BGR2HSV)

        lower_white = np.array([0, 0, 200], dtype=np.uint8)
        upper_white = np.array([180, 50, 255], dtype=np.uint8)
        white_mask = cv2.inRange(hsv, lower_white, upper_white)

        kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5, 5))
        white_mask = cv2.morphologyEx(white_mask, cv2.MORPH_OPEN, kernel)
        white_mask = cv2.morphologyEx(white_mask, cv2.MORPH_CLOSE, kernel, iterations=2)

        contours, _ = cv2.findContours(white_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        if not contours:
            raise ValueError("Could not find any white area in the second image.")

        largest_contour = max(contours, key=cv2.contourArea)

        clean_mask = np.zeros_like(white_mask)
        cv2.drawContours(clean_mask, [largest_contour], -1, 255, thickness=cv2.FILLED)

        return clean_mask

    def _find_dark_lines_mask(self, img_bgra):
        """
        Identifies continuous dark lines in a BGRA image and returns a binary mask.
        """
        # Convert to grayscale
        gray = cv2.cvtColor(img_bgra, cv2.COLOR_BGRA2GRAY)

        # Apply adaptive thresholding to identify dark regions
        thresh = cv2.adaptiveThreshold(gray, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
                                       cv2.THRESH_BINARY_INV, 11, 2)

        # Define a kernel for morphological operations
        kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (3, 3))

        # Apply closing to connect close dark regions
        closed = cv2.morphologyEx(thresh, cv2.MORPH_CLOSE, kernel)
        # Apply opening to remove small noisy areas
        opened = cv2.morphologyEx(closed, cv2.MORPH_OPEN, kernel)

        # Find contours
        contours, _ = cv2.findContours(opened, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

        # Create an empty mask to draw the significant dark lines
        dark_lines_mask = np.zeros_like(gray, dtype=np.uint8)

        # Iterate through contours and filter by area, then draw
        min_contour_area = 50 # Adjust this value based on expected line thickness/size
        for contour in contours:
            if cv2.contourArea(contour) > min_contour_area:
                cv2.drawContours(dark_lines_mask, [contour], -1, 255, thickness=cv2.FILLED)

        return dark_lines_mask

    def process(self, complete_image_path, missing_image_path, output_path, save_intermediate=False, show_matches=False, draw_contours=False, save_svg_contour=False): # Added save_svg_contour
        """
        Main pipeline function.
        """
        complete_image = cv2.imread(complete_image_path, cv2.IMREAD_UNCHANGED)
        missing_image = cv2.imread(missing_image_path, cv2.IMREAD_UNCHANGED)

        if complete_image is None or missing_image is None:
            raise FileNotFoundError("Could not load one or both input images. Check your file paths.")

        if complete_image.shape[2] == 3:
            complete_image = cv2.cvtColor(complete_image, cv2.COLOR_BGR2BGRA)
        if missing_image.shape[2] == 3:
            missing_image = cv2.cvtColor(missing_image, cv2.COLOR_BGR2BGRA)

        if self.debug and show_matches:
            plt.figure(figsize=(20, 10))

            plt.subplot(1, 2, 1)
            plt.imshow(cv2.cvtColor(complete_image, cv2.COLOR_BGRA2RGBA))
            plt.title("Complete Image (Original)")
            plt.axis('off')

            plt.subplot(1, 2, 2)
            plt.imshow(cv2.cvtColor(missing_image, cv2.COLOR_BGRA2RGBA))
            plt.title("Missing Image (Original)")
            plt.axis('off')
            plt.show()

        # Find white region in missing_image and make it transparent
        white_region_mask = self._find_white_region(missing_image)
        missing_image[white_region_mask == 255, 3] = 0

        # Find dark lines in missing_image
        dark_lines_mask = self._find_dark_lines_mask(missing_image)

        gray_complete_image = cv2.cvtColor(complete_image, cv2.COLOR_BGRA2GRAY)
        gray_missing_image = cv2.cvtColor(missing_image, cv2.COLOR_BGRA2GRAY)

        # Create a valid mask for missing_image features: not transparent AND not dark lines
        valid_missing_image_mask = (missing_image[:, :, 3] > 0).astype(np.uint8) * 255
        valid_missing_image_mask = cv2.bitwise_and(valid_missing_image_mask, cv2.bitwise_not(dark_lines_mask))

        # Feature detection
        kp_complete_image, des_complete_image = self.detector.detectAndCompute(gray_complete_image, None)
        kp_missing_image, des_missing_image = self.detector.detectAndCompute(gray_missing_image, mask=valid_missing_image_mask)

        if self.debug:
            print(f"Keypoints found in complete_image: {len(kp_complete_image) if kp_complete_image is not None else 0}")
            print(f"Keypoints found in missing_image (after masking): {len(kp_missing_image) if kp_missing_image is not None else 0}")

        if des_complete_image is None or des_missing_image is None or len(des_complete_image) < 4 or len(des_missing_image) < 4:
            raise ValueError("Not enough features found to align images. Check image content or mask parameters.")

        # Feature matching using knnMatch and ratio test
        matches = self.matcher.knnMatch(des_missing_image, des_complete_image, k=2)

        good_matches = []
        # Use the ratio_thresh that worked well for SIFT BFMatcher (0.75)
        for m, n in matches:
            if m.distance < 0.75 * n.distance:
                good_matches.append(m)

        if self.debug and show_matches:
            # Draw only good matches for diagnostic
            img_matches = cv2.drawMatches(gray_missing_image, kp_missing_image, gray_complete_image, kp_complete_image, good_matches, None, flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)
            plt.figure(figsize=(15, 8))
            plt.imshow(cv2.cvtColor(img_matches, cv2.COLOR_BGR2RGB))
            plt.title("Good Matches (BFMatcher with Ratio Test)")
            plt.savefig('replacement_matching.png')
            plt.show()

        if self.debug:
            print(f"Total knn matches (before ratio test): {len(matches)}")
            print(f"Good matches used for transformation (after ratio test): {len(good_matches)}")

        if len(good_matches) < 4:
            raise ValueError(f"Not enough good matches found to calculate transformation. Found: {len(good_matches)}")

        # Estimate affine transformation
        src_pts = np.float32([kp_missing_image[m.queryIdx].pt for m in good_matches]).reshape(-1, 1, 2)
        dst_pts = np.float32([kp_complete_image[m.trainIdx].pt for m in good_matches]).reshape(-1, 1, 2)

        matrix, inliers = cv2.estimateAffinePartial2D(src_pts, dst_pts, method=cv2.RANSAC)

        if matrix is None:
            raise ValueError("Transformation calculation failed. Images may be too dissimilar.")

        h1, w1 = complete_image.shape[:2]
        # Warp aligned_missing_image to align with complete_image's coordinate system
        aligned_missing_image = cv2.warpAffine(missing_image, matrix, (w1, h1), flags=cv2.INTER_LINEAR, borderMode=cv2.BORDER_CONSTANT, borderValue=(0,0,0,0))

        # Warp the white region mask to get its new aligned position
        aligned_white_mask = cv2.warpAffine(white_region_mask, matrix, (w1, h1), flags=cv2.INTER_NEAREST)

        # --- Generate the output image containing only pixels from complete_image within the mask ---
        # Create a blank BGRA image (transparent)
        final_output = np.zeros_like(complete_image, dtype=np.uint8)

        # Copy pixels from complete_image only where aligned_white_mask is 255
        # Set alpha channel to opaque for these copied pixels
        final_output[aligned_white_mask == 255] = complete_image[aligned_white_mask == 255]
        final_output[aligned_white_mask == 255, 3] = 255 # Ensure these pixels are opaque

        # Optionally draw a sharp red outline around the aligned white region
        contours, _ = cv2.findContours(aligned_white_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
        if draw_contours:
            # Draw red outline, 1 pixel thick (BGRA: 0, 0, 255, 255 for red with full alpha)
            cv2.drawContours(final_output, contours, -1, (0, 0, 255, 255), thickness=1)

        # --- SVG Contour Export ---
        if save_svg_contour:
            svg_filename = os.path.splitext(output_path)[0] + "_contour.svg"
            # Use explicit pixel units for size and correct (width, height) order
            dwg = svgwrite.Drawing(svg_filename, size=(f"{w1}px", f"{h1}px"), profile='tiny')

            for contour in contours:
                # Reshape (N, 1, 2) contour to (N, 2) and convert to list of tuples
                points = contour.reshape(-1, 2).tolist()
                if len(points) > 1: # Polyline requires at least 2 points
                    dwg.add(dwg.polyline(points,
                                         stroke='black',
                                         fill='none',
                                         stroke_width='hairline'))
            dwg.save()
            if self.debug:
                print(f"SVG contour saved to {svg_filename}")

        # Get DPI information from original complete image
        pil_complete_image_info = Image.open(complete_image_path).info
        dpi = pil_complete_image_info.get('dpi', (300, 300))

        # Save the final output PNG
        final_pil = Image.fromarray(cv2.cvtColor(final_output, cv2.COLOR_BGRA2RGBA))
        final_pil.save(output_path, "PNG", dpi=dpi)

        if save_intermediate:
            inter_path = os.path.splitext(output_path)[0] + "_layers.tiff"
            pil_complete_image = Image.fromarray(cv2.cvtColor(complete_image, cv2.COLOR_BGRA2RGBA))
            pil_missing_image = Image.fromarray(cv2.cvtColor(aligned_missing_image, cv2.COLOR_BGRA2RGBA))
            pil_contours = Image.fromarray(cv2.cvtColor(contours, cv2.COLOR_BGRA2RGBA))
            pil_final_output_layer = Image.fromarray(cv2.cvtColor(final_output, cv2.COLOR_BGRA2RGBA))
            pil_complete_image.save(inter_path, format="TIFF", save_all=True, append_images=[pil_missing_image, pil_final_output_layer], dpi=dpi)

        return complete_image, missing_image, final_output

# ==========================================
# Colab Execution Example (Cleaned up)
# ========================================== # 1. Upload your images to the Colab files section (left menu).
# 2. Change the filenames below to match your uploaded files.

if __name__ == "__main__":
    COMPLETE_IMAGE_PATH = "complete.jpg"  # Replace with your image 1 filename
    MISSING_IMAGE_PATH = "missing.jpg"     # Replace with your image 2 filename
    OUTPUT_PATH = "replacement.png"

    # Initialize ImageAligner with default SIFT/BFMatcher
    # Set debug to True for diagnostic prints, False for production
    aligner = ImageAligner(debug=True)

    try:
        # Check if files exist before running
        if os.path.exists(COMPLETE_IMAGE_PATH) and os.path.exists(MISSING_IMAGE_PATH):
            print("Processing images with SIFT/BFMatcher for alignment...")
            # show_matches=True to display the feature matches visualization
            # draw_contours=True to draw a red 1px outline around the replaced region
            show_matches_option = True # Define a local variable for clarity
            complete_image, missing_image, final_output = aligner.process(COMPLETE_IMAGE_PATH,
                                                                          MISSING_IMAGE_PATH,
                                                                          OUTPUT_PATH,
                                                                          save_intermediate=True,
                                                                          show_matches=show_matches_option,
                                                                          draw_contours=False,
                                                                          save_svg_contour=True) # Enabled SVG export

            if aligner.debug and show_matches_option:
                plt.figure(figsize=(10, 8))
                plt.imshow(cv2.cvtColor(final_output, cv2.COLOR_BGRA2RGBA))
                plt.title("Final Output Image")
                plt.axis('off')
                plt.show()

            print(f"Success! Images processed. You can download '{OUTPUT_PATH}', '{os.path.splitext(OUTPUT_PATH)[0] + '_layers.tiff'}', and '{os.path.splitext(OUTPUT_PATH)[0] + '_contour.svg'}' from the files menu.")
        else:
            print("Please upload your images and update the file paths in the script.")
    except Exception as e:
        print(f"Error: {str(e)}")

Output hidden; open in https://colab.research.google.com to view.

In [3]:
import cv2
import numpy as np
import os
from PIL import Image
import matplotlib.pyplot as plt
import svgwrite # Added for SVG output
import argparse # Added for CLI functionality

class ImageAligner:
    """
    Library class for aligning images, isolating regions, and extracting composites.
    """
    def __init__(self, debug=False, detector=None, matcher=None):
        self.debug = debug

        # Initialize detector: SIFT as default as it worked well in tests
        if detector is None:
            self.detector = cv2.SIFT_create()
        else:
            self.detector = detector

        # Initialize matcher: BFMatcher with NORM_L2 as default for SIFT
        if matcher is None:
            self.matcher = cv2.BFMatcher(cv2.NORM_L2, crossCheck=False) # crossCheck=False for knnMatch with ratio test
        else:
            self.matcher = matcher

    def _find_white_region(self, img_bgra):
        """
        Finds the largest white region, ignoring noise and damaged lines.
        Returns a binary mask of the white area.
        """
        hsv = cv2.cvtColor(img_bgra, cv2.COLOR_BGRA2BGR)
        hsv = cv2.cvtColor(hsv, cv2.COLOR_BGR2HSV)

        lower_white = np.array([0, 0, 200], dtype=np.uint8)
        upper_white = np.array([180, 50, 255], dtype=np.uint8)
        white_mask = cv2.inRange(hsv, lower_white, upper_white)

        kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5, 5))
        white_mask = cv2.morphologyEx(white_mask, cv2.MORPH_OPEN, kernel)
        white_mask = cv2.morphologyEx(white_mask, cv2.MORPH_CLOSE, kernel, iterations=2)

        contours, _ = cv2.findContours(white_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        if not contours:
            raise ValueError("Could not find any white area in the second image.")

        largest_contour = max(contours, key=cv2.contourArea)

        clean_mask = np.zeros_like(white_mask)
        cv2.drawContours(clean_mask, [largest_contour], -1, 255, thickness=cv2.FILLED)

        return clean_mask

    def _find_dark_lines_mask(self, img_bgra):
        """
        Identifies continuous dark lines in a BGRA image and returns a binary mask.
        """
        # Convert to grayscale
        gray = cv2.cvtColor(img_bgra, cv2.COLOR_BGRA2GRAY)

        # Apply adaptive thresholding to identify dark regions
        thresh = cv2.adaptiveThreshold(gray, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
                                       cv2.THRESH_BINARY_INV, 11, 2)

        # Define a kernel for morphological operations
        kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (3, 3))

        # Apply closing to connect close dark regions
        closed = cv2.morphologyEx(thresh, cv2.MORPH_CLOSE, kernel)
        # Apply opening to remove small noisy areas
        opened = cv2.morphologyEx(closed, cv2.MORPH_OPEN, kernel)

        # Find contours
        contours, _ = cv2.findContours(opened, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

        # Create an empty mask to draw the significant dark lines
        dark_lines_mask = np.zeros_like(gray, dtype=np.uint8)

        # Iterate through contours and filter by area, then draw
        min_contour_area = 50 # Adjust this value based on expected line thickness/size
        for contour in contours:
            if cv2.contourArea(contour) > min_contour_area:
                cv2.drawContours(dark_lines_mask, [contour], -1, 255, thickness=cv2.FILLED)

        return dark_lines_mask

    def process(self, complete_image_path, missing_image_path, output_path, save_intermediate=False, show_matches=False, draw_contours=False, save_svg_contour=False):
        """
        Main pipeline function.
        """
        complete_image = cv2.imread(complete_image_path, cv2.IMREAD_UNCHANGED)
        missing_image = cv2.imread(missing_image_path, cv2.IMREAD_UNCHANGED)

        if complete_image is None or missing_image is None:
            raise FileNotFoundError("Could not load one or both input images. Check your file paths.")

        if complete_image.shape[2] == 3:
            complete_image = cv2.cvtColor(complete_image, cv2.COLOR_BGR2BGRA)
        if missing_image.shape[2] == 3:
            missing_image = cv2.cvtColor(missing_image, cv2.COLOR_BGR2BGRA)

        if self.debug and show_matches:
            plt.figure(figsize=(20, 10))

            plt.subplot(1, 2, 1)
            plt.imshow(cv2.cvtColor(complete_image, cv2.COLOR_BGRA2RGBA))
            plt.title("Complete Image (Original)")
            plt.axis('off')

            plt.subplot(1, 2, 2)
            plt.imshow(cv2.cvtColor(missing_image, cv2.COLOR_BGRA2RGBA))
            plt.title("Missing Image (Original)")
            plt.axis('off')
            plt.show()

        # Find white region in missing_image and make it transparent
        white_region_mask = self._find_white_region(missing_image)
        missing_image[white_region_mask == 255, 3] = 0

        # Find dark lines in missing_image
        dark_lines_mask = self._find_dark_lines_mask(missing_image)

        gray_complete_image = cv2.cvtColor(complete_image, cv2.COLOR_BGRA2GRAY)
        gray_missing_image = cv2.cvtColor(missing_image, cv2.COLOR_BGRA2GRAY)

        # Create a valid mask for missing_image features: not transparent AND not dark lines
        valid_missing_image_mask = (missing_image[:, :, 3] > 0).astype(np.uint8) * 255
        valid_missing_image_mask = cv2.bitwise_and(valid_missing_image_mask, cv2.bitwise_not(dark_lines_mask))

        # Feature detection
        kp_complete_image, des_complete_image = self.detector.detectAndCompute(gray_complete_image, None)
        kp_missing_image, des_missing_image = self.detector.detectAndCompute(gray_missing_image, mask=valid_missing_image_mask)

        if self.debug:
            print(f"Keypoints found in complete_image: {len(kp_complete_image) if kp_complete_image is not None else 0}")
            print(f"Keypoints found in missing_image (after masking): {len(kp_missing_image) if kp_missing_image is not None else 0}")

        if des_complete_image is None or des_missing_image is None or len(des_complete_image) < 4 or len(des_missing_image) < 4:
            raise ValueError("Not enough features found to align images. Check image content or mask parameters.")

        # Feature matching using knnMatch and ratio test
        matches = self.matcher.knnMatch(des_missing_image, des_complete_image, k=2)

        good_matches = []
        # Use the ratio_thresh that worked well for SIFT BFMatcher (0.75)
        for m, n in matches:
            if m.distance < 0.75 * n.distance:
                good_matches.append(m)

        if self.debug and show_matches:
            # Draw only good matches for diagnostic
            img_matches = cv2.drawMatches(gray_missing_image, kp_missing_image, gray_complete_image, kp_complete_image, good_matches, None, flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)
            plt.figure(figsize=(15, 8))
            plt.imshow(cv2.cvtColor(img_matches, cv2.COLOR_BGR2RGB))
            plt.title("Good Matches (BFMatcher with Ratio Test)")
            plt.savefig('replacement_matching.png')
            plt.show()

        if self.debug:
            print(f"Total knn matches (before ratio test): {len(matches)}")
            print(f"Good matches used for transformation (after ratio test): {len(good_matches)}")

        if len(good_matches) < 4:
            raise ValueError(f"Not enough good matches found to calculate transformation. Found: {len(good_matches)}")

        # Estimate affine transformation
        src_pts = np.float32([kp_missing_image[m.queryIdx].pt for m in good_matches]).reshape(-1, 1, 2)
        dst_pts = np.float32([kp_complete_image[m.trainIdx].pt for m in good_matches]).reshape(-1, 1, 2)

        matrix, inliers = cv2.estimateAffinePartial2D(src_pts, dst_pts, method=cv2.RANSAC)

        if matrix is None:
            raise ValueError("Transformation calculation failed. Images may be too dissimilar.")

        h1, w1 = complete_image.shape[:2]
        # Warp aligned_missing_image to align with complete_image's coordinate system
        aligned_missing_image = cv2.warpAffine(missing_image, matrix, (w1, h1), flags=cv2.INTER_LINEAR, borderMode=cv2.BORDER_CONSTANT, borderValue=(0,0,0,0))

        # Warp the white region mask to get its new aligned position
        aligned_white_mask = cv2.warpAffine(white_region_mask, matrix, (w1, h1), flags=cv2.INTER_NEAREST)

        # --- Generate the output image containing only pixels from complete_image within the mask ---
        # Create a blank BGRA image (transparent)
        final_output = np.zeros_like(complete_image, dtype=np.uint8)

        # Copy pixels from complete_image only where aligned_white_mask is 255
        # Set alpha channel to opaque for these copied pixels
        final_output[aligned_white_mask == 255] = complete_image[aligned_white_mask == 255]
        final_output[aligned_white_mask == 255, 3] = 255 # Ensure these pixels are opaque

        # Optionally draw a sharp red outline around the aligned white region
        contours, _ = cv2.findContours(aligned_white_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
        if draw_contours:
            # Draw red outline, 1 pixel thick (BGRA: 0, 0, 255, 255 for red with full alpha)
            cv2.drawContours(final_output, contours, -1, (0, 0, 255, 255), thickness=1)

        # --- SVG Contour Export ---
        if save_svg_contour:
            svg_filename = os.path.splitext(output_path)[0] + "_contour.svg"
            # Use explicit pixel units for size and correct (width, height) order
            dwg = svgwrite.Drawing(svg_filename, size=(f"{w1}px", f"{h1}px"), profile='tiny')

            # Simplify the contour using approxPolyDP
            # We use the contours found above, which are already from aligned_white_mask and CHAIN_APPROX_NONE
            for contour in contours:
                epsilon = 0.005 * cv2.arcLength(contour, True) # 0.5% of arc length
                approx_contour = cv2.approxPolyDP(contour, epsilon, True)

                # Reshape (N, 1, 2) contour to (N, 2) and convert to list of tuples
                points = approx_contour.reshape(-1, 2).tolist()
                if len(points) > 1: # Polyline requires at least 2 points
                    path_data = f"M {points[0][0]},{points[0][1]}"
                    for p in points[1:]:
                        path_data += f" L {p[0]},{p[1]}"
                    path_data += " Z" # Close the path
                    dwg.add(dwg.path(d=path_data,
                                     stroke='black',
                                     fill='none',
                                     stroke_width='hairline'))
            dwg.save()
            if self.debug:
                print(f"SVG contour saved to {svg_filename}")

        # Get DPI information from original complete image
        pil_complete_image_info = Image.open(complete_image_path).info
        dpi = pil_complete_image_info.get('dpi', (300, 300))

        # Save the final output PNG
        final_pil = Image.fromarray(cv2.cvtColor(final_output, cv2.COLOR_BGRA2RGBA))
        final_pil.save(output_path, "PNG", dpi=dpi)

        if save_intermediate:
            inter_path = os.path.splitext(output_path)[0] + "_layers.tiff"
            pil_complete_image = Image.fromarray(cv2.cvtColor(complete_image, cv2.COLOR_BGRA2RGBA))
            pil_missing_image = Image.fromarray(cv2.cvtColor(aligned_missing_image, cv2.COLOR_BGRA2RGBA))
            # Add the new final_output as the third layer (making it the 4th layer overall)
            pil_final_output_layer = Image.fromarray(cv2.cvtColor(final_output, cv2.COLOR_BGRA2RGBA))
            pil_complete_image.save(inter_path, format="TIFF", save_all=True, append_images=[pil_missing_image, pil_final_output_layer], dpi=dpi)

        return complete_image, missing_image, final_output

# ========================================== # Added main function for CLI
def main():
    parser = argparse.ArgumentParser(description='Align two images, replacing a white region in the second with a part of the first.')
    parser.add_argument('complete_image_path', type=str, help='Path to the complete image (Image 1).')
    parser.add_argument('missing_image_path', type=str, help='Path to the missing image (Image 2).')
    parser.add_argument('output_path', type=str, help='Path to save the final output image (e.g., replacement.png).')
    parser.add_argument('--save_intermediate', action='store_true', help='Save intermediate aligned images in a multi-layer TIFF format.')
    parser.add_argument('--show_matches', action='store_true', help='Display feature matching visualization using matplotlib.')
    parser.add_argument('--draw_contours', action='store_true', help='Draw a red 1px outline around the replaced region in the final output.')
    parser.add_argument('--save_svg_contour', action='store_true', help='Save the aligned contour of the white region as an SVG file.')
    parser.add_argument('--debug', action='store_true', help='Enable debug prints for diagnostic information.')

    args = parser.parse_args()

    aligner = ImageAligner(debug=args.debug) # Pass debug argument to ImageAligner

    try:
        # Check if files exist before running
        if os.path.exists(args.complete_image_path) and os.path.exists(args.missing_image_path):
            print("Processing images with SIFT/BFMatcher for alignment...")
            complete_image, missing_image, final_output = aligner.process(
                args.complete_image_path,
                args.missing_image_path,
                args.output_path,
                save_intermediate=args.save_intermediate,
                show_matches=args.show_matches,
                draw_contours=args.draw_contours,
                save_svg_contour=args.save_svg_contour
            )

            if aligner.debug and args.show_matches:
                plt.figure(figsize=(10, 8))
                plt.imshow(cv2.cvtColor(final_output, cv2.COLOR_BGRA2RGBA))
                plt.title("Final Output Image")
                plt.axis('off')
                plt.show()

            outputs = [f"'{args.output_path}'"]
            if args.save_intermediate:
                outputs.append(f"'{os.path.splitext(args.output_path)[0] + '_layers.tiff'}'")
            if args.save_svg_contour:
                outputs.append(f"'{os.path.splitext(args.output_path)[0] + '_contour.svg'}'")

            print(f"Success! Images processed. You can download {', '.join(outputs)} from the files menu.")
        else:
            print("Please ensure both input image files exist at the specified paths.")
    except Exception as e:
        print(f"Error: {str(e)}")

if __name__ == "__main__":
    main()


usage: colab_kernel_launcher.py [-h] [--save_intermediate] [--show_matches]
                                [--draw_contours] [--save_svg_contour]
                                [--debug]
                                complete_image_path missing_image_path
                                output_path
colab_kernel_launcher.py: error: the following arguments are required: missing_image_path, output_path
ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/lib/python3.12/argparse.py", line 1943, in _parse_known_args2
    namespace, args = self._parse_known_args(args, namespace, intermixed)
                      ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/lib/python3.12/argparse.py", line 2230, in _parse_known_args
    raise ArgumentError(None, _('the following arguments are required: %s') %
argparse.ArgumentError: the following arguments are required: missing_image_path, output_path

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/IPython/core/interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/tmp/ipython-input-1347178740.py", line 293, in <cell line: 0>
    main()
  File "/tmp/ipython-input-1347178740.py", line 255, in main
    args = parser.parse_args()
           ^^^^^^^^^^^^^^^^^^^
  File "/usr/lib/python3.1

TypeError: object of type 'NoneType' has no len()